Purpose: Turn text-data into data with appropriate values neccessary for functional RAGAS evaluation

Fetch documents (from only the SciComp wiki as of now) and put into accessable format for generation of expected outputs and creation of context later on

In [ ]:
# from langchain_community.document_loaders import TextLoader
# import os
# import pandas as pd

# DONE: Recursively load all scraped files in the directory and its subdirectories
loader = TextLoader("../data/test/test.txt")
documents = loader.load()
print(documents)

# from langchain_community.document_loaders import TextLoader
# import os
# import pandas as pd
# import glob

# import glob
# import os
# import json
# import mimetypes
# from bs4 import BeautifulSoup

# class RecursiveTextLoader:
#     def __init__(self, root_dir):
#         self.root_dir = root_dir

#     def load(self):
#         documents = []
#         for filepath in glob.glob(os.path.join(self.root_dir, '**'), recursive=True):
#             if os.path.isfile(filepath):
#                 mime_type, _ = mimetypes.guess_type(filepath)
#                 if mime_type == 'application/json':
#                     with open(filepath, 'r', encoding='utf-8') as file:
#                         documents.append(json.load(file))
#                 elif mime_type == 'text/html' or self._looks_like_html(filepath):
#                     with open(filepath, 'r', encoding='utf-8') as file:
#                         soup = BeautifulSoup(file, 'html.parser')
#                         documents.append(soup.get_text())
#                 elif mime_type == 'text/plain':
#                     with open(filepath, 'r', encoding='utf-8') as file:
#                         documents.append(file.read())
#         return documents

#     def _looks_like_html(self, filepath):
#         with open(filepath, 'r', encoding='utf-8') as file:
#             head = file.read(1024)  # Read the first 1024 bytes
#             return '<html' in head or '<!DOCTYPE html>' in head

# # Assuming your data folder is at "../data/"
# loader = RecursiveTextLoader("../data/")
# documents = loader.load()
# # Assuming documents is a list of strings or convertible to string
# with open('output.txt', 'a', encoding='utf-8') as file:
#     for document in documents:
#         # Remove all linebreaks from the document and then append
#         file.write(str(document).replace('\n', '') + '\n')


Generate the testset

In [ ]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import os
# generator with openai models
from dotenv import load_dotenv
import os

load_dotenv()  # Load .env file
api_key = os.environ.get("OPENAI_API_KEY")

#change the model to a better one once the whole thing is functional

generator_llm = ChatOpenAI(model="gpt-3.5-turbo", api_key=api_key)
# critic_llm = ChatOpenAI(model="gpt-4")
critic_llm = ChatOpenAI(model="gpt-3.5-turbo", api_key=api_key)
embeddings = OpenAIEmbeddings(api_key=api_key)

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
) 


testset = generator.generate_with_langchain_docs(documents, test_size=10, distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25})

Address potentiall duplicate questions (which the testset generator has done) and delete their rows

In [ ]:
df = testset.to_pandas()
df = df.drop_duplicates(subset='question', keep='first')
questions_list = df['question'].tolist()
print (questions_list)

seen = set()
questions_list = [x for x in questions_list if not (x in seen or seen.add(x))]

for item in questions_list:
    print(item)

    

# testset.to_json("testset.json")
# Creates dataset of ground truths contexts and questions for the testset
# Missing answers column 
# On one medium size document, it took about 1 minutes to generate 10 questions and cost 3 dollars on OpenAI
# Seems expensive when using gpt-4, is its use justified or does 3.5 get the job done?
# Evaluate gpt-4 vs gpt-3.5-turbo-16k for RAGAS evaluation test data generation


In [ ]:
df.to_csv('testset.csv', index=False)

fetch the LLM's response and append to a list

In [ ]:
# Assuming SemanticSearchParser is your class and it has a method named 'generate_response' that takes a question and returns an answer

# Initialize your SemanticSearchParser class
# Adjust this step if your class initialization requires different parameters


# Now you can import the class

from generate_answer import SemanticSearchService

weaviate_url = "http://localhost:8777"
service = SemanticSearchService(weaviate_url)
print (questions_list)

# List to store answers (optional)
answers_list = []


# Loop through each question in the questions_list
for question in questions_list:
    # Use the question as input to get the answer
    answer = service.generate_response(question)
    
    # Print the answer
    
    # Optionally, append the answer to answers_list for further processing
    answers_list.append(answer)

print (answers_list)
temp_ans_list = ["The Janelia Scientific Computing team operates and maintains a world-class computational infrastructure that includes a high-performance compute cluster with over 5000 cores and 300 GPUs. This infrastructure is used to analyze and mine the large amounts of data produced by Janelia's scientists. The team also supports a state-of-the-art storage and compute infrastructure across two data centers, which currently supports over 15 petabytes of scientific data. This data is split across various storage tiers and is connected with an optical fiber ring. The team also maintains a 4500 sq ft data center with significant power and cooling capacity. \n\nIn addition to hardware, the team also has deep software skills in a broad range of programming languages, extendable applications, frameworks, cloud & cluster technologies, and databases. These skills are used to help with research and engineering tasks, from quick questions to full software life cycle support. The team's software skills, combined with their domain knowledge in areas such as image processing, machine learning, data handling, microscopy, instrument control, 3D graphics & visualization, and bioinformatics & transcriptomics, allow them to efficiently work with both experimentalists and computer scientists. \n\nThe team also develops and maintains a variety of tools and projects, such as NeuronBridge, HortaCloud, VVD Viewer, EASI-FISH pipeline, Render, RS-FISH, and BigStitcher, which are used for various aspects of data analysis and simulation in biological research.", "The Janelia Scientific Computing team provides a wide range of support for advanced imaging techniques and image analysis. They offer consultation on experiment design as well as image visualization and processing. They also provide comprehensive image and data analysis support for multiple software packages through hands-on assistance and/or custom-written macros/plugins/scripts for ImageJ/FIJI, MATLAB, Imaris, etc. \n\nIn addition, they maintain several computer workstations dedicated to viewing and processing large image datasets acquired with the facility's instruments. These workstations are equipped with a suite of imaging software, including a full version of Imaris, and have robust hardware specifications to handle large datasets. \n\nThe team also has deep domain knowledge in image processing, machine learning, data handling, and 3D graphics & visualization, which allows them to efficiently work with experimentalists and computer scientists in various research areas.", "The Janelia Scientific Computing team provides world-class computational infrastructure to support the institute's scientific endeavors. They operate and maintain all of Janelia’s storage and associated backup infrastructure, high performance compute cluster, and all Linux systems. They also manage Janelia’s data center and backup and disaster recovery resources. The team supports a Linux compute cluster with over 5000 cores and 300 GPUs, and is responsible for maintaining many other Linux servers and workstations. They also handle a significant amount of data, with almost 100TB of new Janelia’s data being safely backed up every month.\n\nIn addition to infrastructure, the Scientific Computing Software team works closely with Janelia's labs and project teams, providing everything from answering quick questions to full software life cycle support. They have a broad range of software skills, including programming languages, extendable applications, frameworks, cloud & cluster technologies, and databases. They also have deep domain knowledge in areas like image processing, machine learning, data handling, microscopy, instrument control, 3D graphics & visualization, and bioinformatics & transcriptomics. \n\nThe team also identifies opportunities for code reuse, reducing development overhead and support costs across Janelia. They are strong proponents of open science and have created the Open Science Software Initiative. Most of their software is open source and available via GitHub. They also run a Scientific Computing Associates program to embed associates in SciComp and the lab or team they work with. \n\nThe team is led by Stephan Preibisch and consists of three teams: Software Engineering headed by Konrad Rokicki, Computational Methods and Solutions, both headed by Stephan Preibisch. They have developed several tools and projects like NeuronBridge, HortaCloud, VVD Viewer, EASI-FISH pipeline, Render, RS-FISH, and BigStitcher. \n\nIn summary, the Janelia Scientific Computing team supports the institute's mission and drives innovation in modern biological research by providing robust computational infrastructure, software support, and developing innovative tools and solutions.", 'The Janelia Scientific Computing team supports advanced imaging techniques in neuroscience and cell biology through a variety of ways. They have deep domain knowledge in image processing, machine learning, data handling, electron and light microscopy, instrument control, 3D graphics & visualization, bioinformatics & transcriptomics. They also develop and maintain a range of software tools and applications that aid in these areas. Some of these tools include NeuronBridge for finding neuron matches across modalities, HortaCloud for cloud-based collaborative annotation, VVD Viewer for volumetric rendering of 3D/4D microscopy data, and BigStitcher for efficient alignment of multi-tile and multi-angle image datasets. They also work closely with labs and project teams, providing full software life cycle support.', "The Janelia Scientific Computing team supports modern biological research in several ways. They maintain a world-class computational infrastructure, including storage and backup infrastructure, a high-performance compute cluster, and all Linux systems. They also manage Janelia's data center and backup and disaster recovery resources. The team supports data storage infrastructure for storing and accessing scientific data, with over 15 petabytes of scientific data split across various storage tiers. They also support a Linux compute cluster with over 5000 cores and 300 GPUs, and maintain many other Linux servers and workstations. \n\nIn addition to infrastructure support, the Scientific Computing Software team works closely with Janelia's labs, project teams, and shared resources to help with research and engineering tasks. They provide everything from answering quick questions to full software life cycle support. The team's software skills span a broad range of programming languages, extendable applications, frameworks, cloud & cluster technologies, and databases. They also have deep domain knowledge in image processing, machine learning, data handling, electron and light microscopy, instrument control, 3D graphics & visualization, bioinformatics & transcriptomics. \n\nThe team also develops and maintains a variety of tools and projects, such as NeuronBridge, HortaCloud, VVD Viewer, EASI-FISH pipeline, Render, RS-FISH, and BigStitcher. They are strong proponents of open science and most of their software is open source and available via GitHub. They also run the Scientific Computing Associates program, which offers challenging assignments for those interested in computational science.", "The Janelia Scientific Computing team provides comprehensive support for advanced imaging techniques in neuroscience, cell biology, and bioinformatics through a variety of collaborations and custom software tools. They work closely with Janelia's labs, project teams, and shared resources to assist with research and engineering tasks. This can range from answering quick questions to providing full software life cycle support.\n\nThe team's software skills cover a broad range of programming languages, extendable applications, frameworks, cloud & cluster technologies, and databases. They have deep domain knowledge in image processing, machine learning, data handling, electron and light microscopy, instrument control, 3D graphics & visualization, bioinformatics & transcriptomics. Many team members have backgrounds in biology, enabling them to work efficiently with experimentalists and computer scientists.\n\nThe team is also involved in various projects and tools such as NeuronBridge, HortaCloud, VVD Viewer, EASI-FISH pipeline, Render, RS-FISH, and BigStitcher, which are designed to support advanced imaging techniques and data analysis in neuroscience, cell biology, and bioinformatics.\n\nFurthermore, the team is a strong proponent of open science and has teamed up with the Computation & Theory research area to create the Open Science Software Initiative. Most of their software is open source and available via GitHub, promoting collaboration and knowledge sharing. They also run the Scientific Computing Associates program, which embeds associates in SciComp and the lab or team they work with."]




Add values from the list of JaneliaGPT responses to the dataframe

In [ ]:
df['answer'] = None
# Assuming df is your DataFrame and answers_list is a list with values to populate the 'Answer' column
if len(df) == len(answers_list):
    df['answer'] = answers_list
else:
    print("The length of answers_list does not match the number of rows in the DataFrame.")

df.dropna(axis=1, how='all', inplace=True)


In [ ]:


display(df)
df.to_json('WithAnswersDatasetFromTestTxt.json', index=True)

In [ ]:
#Attempt to fix the issue with the answers_list not bsjdkflasdjjfklasdkl fasdjkfh jkas
df_fix = df
# List of columns to keep
columns_to_keep = ['question', 'ground_truth', 'answer', 'contexts']

# Reassign df to a DataFrame containing only the columns to keep
df_fix= df_fix[columns_to_keep]

display(df_fix['contexts'])

In [ ]:
# Assuming df is already defined and contains the necessary columns

# Convert 'question' and 'answer' to lists of strings if they are not already
df_fix['question'] = df_fix['question'].apply(lambda x: [x] if isinstance(x, str) else x)
df_fix['answer'] = df_fix['answer'].apply(lambda x: [x] if isinstance(x, str) else x)

# Ensure 'contexts' and 'ground_truth' are lists of lists of strings
# This step assumes 'contexts' and 'ground_truth' are already in the correct format
# If not, you would need to apply a similar conversion as above, ensuring each element is a list

# Example conversion if 'contexts' and 'ground_truth' were not already lists of lists
df_fix['contexts'] = df_fix['contexts'].apply(lambda x: [[y] for y in x] if isinstance(x, list) and all(isinstance(y, str) for y in x) else x)
df_fix['ground_truth'] = df_fix['ground_truth'].apply(lambda x: [[y] for y in x] if isinstance(x, list) and all(isinstance(y, str) for y in x) else x)

# Now df should be in the correct format for training

In [ ]:
#  This converts to dictionary, dont think we need this right now

# # Select the first row of the DataFrame (assuming it's the one you want to convert)
# row = df_fix.iloc[0]

# # Convert the selected row to a dictionary
# result_dict = {
#     'question': row['question'][0] if row['question'] else None,  # Assuming 'question' is a list of strings
#     'answer': row['answer'][0] if row['answer'] else None,  # Assuming 'answer' is a list of strings
#     'contexts': row['contexts'] if row['contexts'] else [],  # Assuming 'contexts' is already a list of strings
#     'ground_truths': row['ground_truth'] if row['ground_truth'] else []  # Renaming key and assuming it's a list of lists
# }
# print (result_dict)
# # result_dict now resembles the desired format

# formatted_dataset = {
#     'features': list(result_dict.keys()),  # List of keys from result_dict
#     'num_rows': 1  # Since we're only dealing with a single row
# }

# print(formatted_dataset)

In [ ]:
from datasets import load_dataset
import os

amnesty_qa = load_dataset("explodinggradients/amnesty_qa", "english_v2")
display (amnesty_qa["eval"])
display(df_fix)
#  print (formatted_dataset)

Run the evaluation given the dataframe (does not work)

In [ ]:
from datasets import Dataset
dataset_fix = Dataset.from_pandas(df_fix)
dataset_fix = dataset_fix.remove_columns(['__index_level_0__'])

print (dataset_fix)
print (amnesty_qa["eval"])

In [ ]:
from datasets import Features

# Assuming dataset_fix and amnesty_qa["eval"] are your datasets

# Access the Features of each dataset
features_dataset_fix = dataset_fix.features
features_amnesty_eval = amnesty_qa["eval"].features
def format_columns(example):
    # Format 'question', 'answer', and 'ground_truths' columns to single values
    for column in ['question', 'answer', 'ground_truths']:
        if column in example and example[column]:
            example[column] = example[column][0]
    
    # Correctly format 'contexts' column to a list of list of strings
    if 'contexts' in example:
        # Ensure 'contexts' is a list of strings (not a list of lists)
        if isinstance(example['contexts'], list):
            # If the items are lists (or other non-string types), flatten and convert to strings
            example['contexts'] = [str(item) for sublist in example['contexts'] for item in (sublist if isinstance(sublist, list) else [sublist])]
        else:
            # If 'contexts' is not a list, convert it into a list of a single string
            example['contexts'] = [str(example['contexts'])]

    
    return example


# Apply the transformation to both datasets
dataset_fix = dataset_fix.map(format_columns)
# Direct comparison of data types
if set(features_dataset_fix.keys()) == set(features_amnesty_eval.keys()):
    all_types_match = True
    for key in features_dataset_fix.keys():
        type_dataset_fix = type(features_dataset_fix[key]).__name__
        type_amnesty_eval = type(features_amnesty_eval[key]).__name__
        if type_dataset_fix != type_amnesty_eval:
            print(f"Data type for feature '{key}' differs between datasets. dataset_fix: {type_dataset_fix}, amnesty_qa['eval']: {type_amnesty_eval}")
            all_types_match = False
    if all_types_match:
        print("The data types of all features in both datasets match.")
else:
    print("The Features of the datasets differ in their keys.")

In [ ]:



from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)


from ragas import evaluate


# Will reutrn a dataframe with the metrics
# Returns error for now because answers column is missing
# Error is misleading, fix dataset first and make it match the docs example dataset

result = evaluate(
    dataset_fix,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
)

result

result.to_pandas()
